In [17]:
import pandas as pd
import spacy
import random 
import json
import os
from pathlib import Path
from spacy.training import Example
from spacy.tokens import DocBin
from spacy.language import Language
from spacy.tokens import Span
from spacy.pipeline import EntityRuler
from sklearn.model_selection import train_test_split



os.chdir(r'/home/dlin1765/repos/tekken-move-guesser')
with open('annotations.json', 'r') as raw:
    data = json.load(raw)


In [31]:
import json
from types import NoneType
# create new training data 
limbList = ['punch', 'jab', 'palm', 'fist', 'uppercut', 'hand', 'chop', 'slap', 'armbar', 'swing', 'shove', 'push', 'elbow', 'shoulder', 'tackle', 'body', 'head', 'headbutt', 'finger', 'butt', 'leg', 'foot', 'kick', 'roundhouse', 'round house', 'axe kick', 'axekick', 'heel', 'sweep', 'slide', 'knee']
specialLimbList = ['sword', 'dagger', 'gun', 'pistol', 'handgun', 'nunchuck', 'cane', 'chainsaw', 'chain saw', 'chain-saw', 'magic', 'energy', 'flip', 'backflip', 'cartwheel', 'cart wheel']
howListPre = ['jumping', 'bouncing', 'hopping','running', 'spinning', 'approaching', 'advancing', 'forward moving', 'backwards moving', 'backwards', 'retreating', 'dodging', 'evading','flying', 'rolling', 'sliding', 'overhead', 'downwards', 'rising', 'forward']
howListDescriptors = ['big', 'long', 'long range', 'giant', 'huge', 'fast', 'slow', 'quick', 'rapid','reactable']
howListPost = ['strike', 'move', 'poke', 'check', 'smash', 'jab', 'move', 'move', 'move', 'move']
specialHowList = ['swing', 'stab', 'blast', 'strike', 'move', 'poke', 'check', 'smash', 'jab', 'move', 'move', 'move', 'move', 'slash', 'beam', 'thrust']

tekkenPropertyDescriptors = ['plus on block', 'plus frame', 'plus', 'minus on block', 'minus', 'unsafe', 'punishable']
tekkenProperties = ['high', 'low', 'mid', 'guard crush', 'unblockable', 'untechable', 'launcher', 'stance', 'tornado', 'knockdown', 'knock down', 'stagger', 'high crush', 'low crush', 'high crushing', 'low crushing', 'evasive', 'homing']
tekkenPropertiesPost = ['guard crushes', 'launches', 'tornados', 'knocks down', 'staggers', 'high crushes', 'low crushes', 'evades']
tekkenMoveTypes = ['heat engager', 'heat smash', 'heat burst', 'parry', 'grab', 'power crush', 'armor', 'armored', 'throw', 'chain throw', 'chain grab']


string_detail = ['two hit', 'three hit', 'four hit', 'five hit', 'multihit', 'two part', 'three part', 'four part', 'five part']
string_detailPost = ['first hit', 'last hit', 'starts with', 'begins with']
actions = ['punches', 'kicks', 'palms', 'jabs', 'uppercuts', 'chops', 'slaps', 'swings', 'pushes', 'shoves', 'elbows', 'knees', 'headbutts', 'butts', 'tackles', 'sweeps', 'slides']
# detail + string + where he 
# detail + string where he/she/they hits you with ()

#maybe save the specific stuff for the specific pipeline or manual entry

# Entities 
# LIMB SPECIAL_LIMB LIMB_HOW LIMB_DESCRIPTION MOVE_TYPE MOVE_PROPERTY STRING_DETAIL

#{"classes": ["LIMB", "SPECIAL_LIMB", "LIMB_HOW", "LIMB_DESCRIPTION", "MOVE_TYPE", "MOVE_PROPERTY", "STRING_DETAIL"], "annotations": null}

def generateStrings2(listOfWords, label1, label2):
    # for each list in the list of words 
    count = 0
    random.seed(10)
    train_data = open("traindata2.txt","a")
    test_data = open('testdata2.txt', "a")
    allAnnotatedLists = []
    for x in range(0, len(listOfWords)):
        for y in range(x+1, len(listOfWords)):
            for word1 in listOfWords[x]:
                for word2 in listOfWords[y]:
                    if(count % 5 != 0):
                        newString = word1 + " " + word2
                        #train_data.write(newString + "\n")
                        labeledList1 = annotateData(0, word1, label1)
                        labeledList2 = annotateData(len(word1) + 1, word2 , label2)
                        annotatedList = [labeledList1, labeledList2]
                        allAnnotatedLists.append([newString, {"entities": annotatedList}])
                    else:
                        newString = word1 + " " + word2 + ' ' + howListPost[random.randrange(0,10)]
                        #train_data.write(newString + "\n")
                        labeledList1 = annotateData(0, word1, label1)
                        labeledList2 = annotateData(len(word1) + 1, word2 , label2)
                        annotatedList = [labeledList1, labeledList2]
                        allAnnotatedLists.append([newString, {"entities": annotatedList}])
                    count += 1
    #addJSONData(allAnnotatedLists)
    addDataToJSONandTxt(allAnnotatedLists)

def generateStrings2a(listOfWords, label1, label2):
    # for each list in the list of words 
    count = 0
    random.seed(10)
    train_data = open("traindata2.txt","a")
    test_data = open('testdata2.txt', "a")
    allAnnotatedLists = []
    for x in range(0, len(listOfWords)):
        for y in range(x+1, len(listOfWords)):
            for word1 in listOfWords[x]:
                for word2 in listOfWords[y]:
                    if(count % 5 != 0):
                        newString = word1 + " that " + word2
                        #train_data.write(newString + "\n")
                        labeledList1 = annotateData(0, word1, label1)
                        labeledList2 = annotateData(len(word1 +" that "), word2 , label2)
                        annotatedList = [labeledList1, labeledList2]
                        allAnnotatedLists.append([newString, {"entities": annotatedList}])
                    else:
                        newString = word1 + " " + howListPost[random.randrange(0,10)] + " that " + word2
                        #train_data.write(newString + "\n")
                        labeledList1 = annotateData(0, word1, label1)
                        labeledList2 = annotateData(len(word1) + 1, word2 , label2)
                        annotatedList = [labeledList1, labeledList2]
                        allAnnotatedLists.append([newString, {"entities": annotatedList}])
                    count += 1
    #addJSONData(allAnnotatedLists)
    addDataToJSONandTxt(allAnnotatedLists)

def generateStrings2String(listOfWords, label1, label2):
    count = 0
    random.seed(10)
    allAnnotatedLists = []
    for x in range(0, len(listOfWords)):
        for y in range(x+1, len(listOfWords)):
            for word1 in listOfWords[x]:
                for word2 in listOfWords[y]:
                    if(count % 5 != 0):
                        newString = word1 + " " + word2 + " string"
                        #train_data.write(newString + "\n")
                        labeledList1 = annotateData(0, word1, label1)
                        labeledList2 = annotateData(len(word1 + " "), word2 , label2)
                        labeledList3 = annotateData(len(word1 + " " + word2 + " "), 'string', 'STRING_DETAIL')
                        annotatedList = [labeledList1, labeledList2, labeledList3]
                        allAnnotatedLists.append([newString, {"entities": annotatedList}])
                    else:
                        newString = word1 + " "  + word2 + " string"
                        #train_data.write(newString + "\n")
                        labeledList1 = annotateData(0, word1, label1)
                        labeledList2 = annotateData(len(word1 + " "), word2 , label2)
                        labeledList3 = annotateData(len(word1 + " " + word2 + " "), 'string', 'STRING_DETAIL')
                        annotatedList = [labeledList1, labeledList2, labeledList3]
                        allAnnotatedLists.append([newString, {"entities": annotatedList}])
                    count += 1
    #addJSONData(allAnnotatedLists)
    addDataToJSONandTxt(allAnnotatedLists)

def generateStrings3(listA, listB, listC, label1, label2, label3):
    count = 0
    random.seed(10)
    allAnnotatedLists = []
    for word1 in listA:
        for word2 in listB:
            for word3 in listC:
                if count % 5 != 0:
                    newString = word1 + " " + word2 + " " + word3
                    labeledList1 = annotateData(0, word1, label1)
                    labeledList2 = annotateData(len(word1) + 1, word2 , label2)
                    labeledList3 = annotateData(len(word1 + " " + word2 + " "), word3, label3)
                    annotatedList = [labeledList1, labeledList2, labeledList3]
                    allAnnotatedLists.append([newString, {"entities": annotatedList}])
                else:
                    newString = word1 + " " + word2 + " " + word3 + " " + howListPost[random.randrange(0,10)]
                    labeledList1 = annotateData(0, word1, label1)
                    labeledList2 = annotateData(len(word1) + 1, word2 , label2)
                    labeledList3 = annotateData(len(word1 + " " + word2 + " "), word3, label3)
                    annotatedList = [labeledList1, labeledList2, labeledList3]
                    allAnnotatedLists.append([newString, {"entities": annotatedList}])
                count += 1
    addDataToJSONandTxt(allAnnotatedLists)

def generateStrings3a(listA, listB, listC, label1, label2, label3):
    count = 0
    random.seed(10)
    allAnnotatedLists = []
    for word1 in listA:
        for word2 in listB:
            for word3 in listC:
                if count % 5 != 0:
                    newString = word1 + " " + word2 + " that " + word3
                    labeledList1 = annotateData(0, word1, label1)
                    labeledList2 = annotateData(len(word1) + 1, word2 , label2)
                    labeledList3 = annotateData(len(word1 + " " + word2 + " that "), word3, label3)
                    annotatedList = [labeledList1, labeledList2, labeledList3]
                    allAnnotatedLists.append([newString, {"entities": annotatedList}]) 
                else:
                    newString = word1 + " " + word2 + " move" +" that " + word3 
                    labeledList1 = annotateData(0, word1, label1)
                    labeledList2 = annotateData(len(word1) + 1, word2 , label2)
                    labeledList3 = annotateData(len(word1 + " " + word2 + " move that "), word3, label3)
                    annotatedList = [labeledList1, labeledList2, labeledList3]
                    allAnnotatedLists.append([newString, {"entities": annotatedList}]) 
                count += 1
    addDataToJSONandTxt(allAnnotatedLists)

def generateStrings3String(listA, listB, listC, label1, label2, label3):
    random.seed(10)
    allAnnotatedLists = []
    for word1 in listA:
        for word2 in listB:
            for word3 in listC:
                newString = word1 + " " + word2 + " " + word3 + " string"
                labeledList1 = annotateData(0, word1, label1)
                labeledList2 = annotateData(len(word1) + 1, word2 , label2)
                labeledList3 = annotateData(len(word1 + " " + word2 + " "), word3, label3)
                labeledList4 = annotateData(len(word1 + " " + word2 + " " + word3 + " "), 'string', 'STRING_DETAIL')
                annotatedList = [labeledList1, labeledList2, labeledList3, labeledList4]
                allAnnotatedLists.append([newString, {"entities": annotatedList}])
    addDataToJSONandTxt(allAnnotatedLists)

def generateStringsStringPre(listA, listB, listC, label1, label2, label3):
    # move/string where he/she/they do a* punch (into a)* punch (into a)* punch
    random.seed(10)
    allAnnotatedLists = []
    stringMove = ['string', 'move']
    pronoun = ['he', 'she', 'they']
    for word1 in listA:
        for word2 in listB:
            for word3 in listC:
                pronounNum = random.randint(0,2)
                if pronounNum == 2:
                    transition = " do a"
                else:
                    transition = " does a"
                newString = stringMove[random.randint(0,1)] + " where " + pronoun[pronounNum] + transition
                
                big1 = randomWordChance(0,1,1,howListDescriptors, "")
                big2 = randomWordChance(0,1,1,howListDescriptors, "")
                big3 = randomWordChance(0,1,1,howListDescriptors, "")

                jumping1 = randomWordChance(0,1,1,howListPre, "")
                jumping2 = randomWordChance(0,1,1,howListPre, "")
                jumping3 = randomWordChance(0,1,1,howListPre, "")

                intoA1 = randomWordChance(0,1,1, ['into a'], "")
                intoA2 = randomWordChance(0,1,1, ['into a', 'into another'], "")
                newString = newString + big1 + jumping1 + " " + word1 + intoA1 + big2 + jumping2 + " " + word2 + intoA2 + big3 + jumping3 + " " + word3
                print(newString)
    #             labeledList1 = annotateData(0, word1, label1)
    #             labeledList2 = annotateData(len(word1) + 1, word2 , label2)
    #             labeledList3 = annotateData(len(word1 + " " + word2 + " "), word3, label3)
    #             labeledList4 = annotateData(len(word1 + " " + word2 + " " + word3 + " "), 'string', 'STRING_DETAIL')
    #             annotatedList = [labeledList1, labeledList2, labeledList3, labeledList4]
    #             allAnnotatedLists.append([newString, {"entities": annotatedList}])
    # addDataToJSONandTxt(allAnnotatedLists)
    # return

def randomWordChance(start = 0, end = 1, desired = 1, wordIfTrue =[], wordIfFalse = ""):
    randNum = random.randint(start, end)
    if randNum == desired:
        return " " + wordIfTrue[random.randint(0, len(wordIfTrue) - 1)]
    else:
        return wordIfFalse

def annotateData(startIndex, newWord, label):
    return [startIndex, startIndex + len(newWord), label]

def updateJSONData(newData):
    with open('annotationsv2.json', 'r') as file:
        data = json.load(file)
        if isinstance(data['annotations'], NoneType):
            tempList = []
            tempList.append(newData)
            data['annotations'] = tempList
        else:
            tempList = []
            for existingList in data['annotations']:
                tempList.append(existingList)
            tempList.append(newData)
            data['annotations'] = tempList
    os.remove('annotationsv2.json')
    with open('annotationsv2.json', 'w') as file:
        json.dump(data, file)

def updateJSONData2(newData):
    with open('annotationstest.json', 'r') as file:
        data = json.load(file)
        if isinstance(data['annotations'], NoneType):
            tempList = []
            tempList.append(newData)
            data['annotations'] = tempList
        else:
            tempList = []
            for existingList in data['annotations']:
                tempList.append(existingList)
            tempList.append(newData)
            data['annotations'] = tempList
    os.remove('annotationstest.json')
    with open('annotationstest.json', 'w') as file:
        json.dump(data, file)

def addJSONData(newData, filename):
    with open(filename, 'r') as file:
        data = json.load(file)
        if isinstance(data['annotations'], NoneType):
            tempList = []
            for x in newData:
                tempList.append(x)
            data['annotations'] = tempList
        else:
            tempList = []
            for existingList in data['annotations']:
                tempList.append(existingList)
            for x in newData:
                tempList.append(x)
            data['annotations'] = tempList
    os.remove(filename)
    with open(filename, 'w') as file:
        json.dump(data, file)

def Testing():
    with open('annotationsv2.json', 'r') as file:
        data = json.load(file)
        print(type(data['annotations']))

def printSimilarityScores(docs, length):
    i = 0
    j = 0
    avg = 0
    while(i < length):
        j = 0
        while(j <  length):
            if i != j:
                avg = avg + docs[i].similarity(docs[j])
                print("Similarity between ", docs[i], " and ", docs[j], docs[i].similarity(docs[j]))
            j += 1
        print('similarity avg = ', avg / length, 'avg total = ', avg)
        avg = 0
        i += 1

def addDataToJSONandTxt(allAnnotatedLists):
    train_data = open("traindata2.txt","a")
    test_data = open('testdata2.txt', "a")
    train, test =  train_test_split(allAnnotatedLists, test_size =.20, shuffle  = True)
    for item in train:
        train_data.write(item[0] + "\n")
    addJSONData(train, 'annotationsv2.json')
    for item in test:
        test_data.write(item[0] + "\n")
    addJSONData(test, 'annotationsTestV2.json')



In [19]:
# testing code block
#generateStrings2Test([howListDescriptors, limbList], 'LIMB_DESCRIPTION', 'LIMB') #.8
#generateStrings2aTest([howListDescriptors, limbList], 'LIMB_DESCRIPTION', 'LIMB')

# this works
# generateStrings2([howListDescriptors, limbList], 'LIMB_DESCRIPTION', 'LIMB')

#{"classes": ["LIMB", "SPECIAL_LIMB", "LIMB_HOW", "LIMB_DESCRIPTION", "MOVE_TYPE", "MOVE_PROPERTY", "STRING_DETAIL"], "annotations": null}


In [20]:
# limbList = ['punch', 'jab', 'palm', 'fist', 'uppercut', 'hand', 'chop', 'slap', 'armbar', 'swing', 'shove', 'push', 'elbow', 'shoulder', 'tackle', 'body', 'head', 'headbutt', 'finger', 'butt', 'leg', 'foot', 'kick', 'roundhouse', 'round house', 'axe kick', 'axekick', 'heel', 'sweep', 'slide', 'knee']
# specialLimbList = ['sword', 'dagger', 'gun', 'pistol', 'handgun', 'nunchuck', 'cane', 'chainsaw', 'chain saw', 'chain-saw', 'magic', 'energy', 'flip', 'backflip', 'cartwheel', 'cart wheel']
# howListPre = ['jumping', 'bouncing', 'hopping','running', 'spinning', 'approaching', 'advancing', 'forward moving', 'backwards moving', 'backwards', 'retreating', 'dodging', 'evading','flying', 'rolling', 'sliding', 'overhead', 'downwards', 'rising', 'forward']
# howListDescriptors = ['big', 'long', 'long range', 'giant', 'huge', 'fast', 'slow', 'quick', 'rapid','reactable']
# howListPost = ['strike', 'move', 'poke', 'check', 'smash', 'jab', 'punch', 'move', 'move', 'move', 'move']
# specialHowList = ['swing', 'stab', 'blast', 'whip', 'move', 'move', 'move', 'move', 'move', 'strike']

# tekkenPropertyDescriptors = ['plus on block', 'plus frame', 'plus', 'minus on block', 'minus', 'unsafe', 'punishable']
# tekkenProperties = ['high', 'low', 'mid', 'guard crush', 'unblockable', 'untechable', 'launcher', 'stance', 'tornado', 'knockdown', 'knock down', 'stagger', 'high crush', 'low crush', 'high crushing', 'low crushing', 'evasive']
# tekkenPropertiesPost = ['guard crushes', 'launches', 'transitions to stance',  'tornados', 'knocks down', 'staggers', 'high crushes', 'low crushes', 'evades']
# tekkenMoveTypes = ['heat engager', 'heat smash', 'heat burst', 'parry', 'grab', 'power crush', 'armor', 'armored', 'throw', 'chain throw', 'chain grab', 'backswing blow', 'hopkick']

# cancan dick jab electric flash getup hayashida hellsweep jackhammer junkyard laser scraper matterhorn otgf omen snake edge

# MOVE TYPES power crushes 

# ENTS (LIMB SPECIAL_LIMB LIMB_HOW LIMB_DESCRIPTION MOVE_TYPE MOVE_PROPERTY FRAMES LOWMIDHIGH STRING_DETAIL)

# generate new strings 
# structure: (descriptor) (limb)
# big punch
generateStrings2([howListDescriptors, limbList], 'LIMB_DESCRIPTION', 'LIMB')

# generate new strings 
# structure: (descriptor) (limb)
# punch that launches
generateStrings2a([limbList, tekkenPropertiesPost], 'LIMB', 'MOVE_PROPERTY')

# structure: pre description limb
# jumping punch
generateStrings2([howListPre, limbList], 'LIMB_HOW', 'LIMB')

# generate new strings 
# structure: (tekken p) (move) that (tekken properties post)
# plus on block punch
generateStrings2([tekkenPropertyDescriptors, limbList], 'MOVE_PROPERTY', 'LIMB')

# generate new strings
# structure: limb limb limb string 
# punch punch string
generateStrings2String([limbList, limbList], 'LIMB', 'LIMB')

# generate new strings
# structure: (descriptor) (pre descriptor) (limb)
# big jumping punch
generateStrings3(howListDescriptors, howListPre, limbList, "LIMB_DESCRIPTION", 'LIMB_HOW', 'LIMB')

# generate new strings
# structure: move/moves where he/she/they goes into stance
# goes / does / brings / does that / 

# generate new strings big punch that x
# structure: (howlistdescriptors) (move) that (tekken properties post)
# big punch that launches 
generateStrings3a(howListDescriptors, limbList, tekkenPropertiesPost, 'LIMB_DESCRIPTION', 'LIMB', 'MOVE_PROPERTY')

# generate new strings
# structure: limb () where he/she/they goes into stance
# sword hilt strike into a sword slash 






# generate new strings
# structure: 
# plus on block jumping punch 
# docs = similar(test)
# docs1 = similar(test2)
# printSimilarityScores(docs1, 4)


In [21]:
# generate new strings
# structure: limb limb limb string 
generateStrings3String(limbList, limbList, limbList, 'LIMB', 'LIMB', 'LIMB')

In [32]:
generateStringsStringPre(limbList, limbList, limbList, 'LIMB', 'LIMB', 'LIMB')

string where they do a quick sliding punch into a approaching punch punch
move where he does a backwards punch into a long range rolling punch slow jab
string where she does a reactable punch into a big punch palm
move where she does a quick punch punch rapid forward moving fist
move where she does a rising punch forward punch uppercut
string where she does a punch slow spinning punch into another quick hand
string where she does a flying punch punch into another fast chop
string where he does a quick sliding punch into a reactable punch overhead slap
move where he does a slow downwards punch into a sliding punch into a rapid armbar
move where he does a punch punch into another long sliding swing
move where they do a huge punch slow punch rapid forward shove
move where they do a punch hopping punch into a slow push
move where they do a approaching punch into a reactable punch giant elbow
move where they do a rapid forward punch quick running punch into another shoulder
string where she

In [ ]:
# generate new strings
# structure: string/move (that goes/starts with) limb limb limb 

# structure: string where he/she/they (do a)/(does a) (big)/(jumping)/(big jumping) punch punch punch

# structure: move/string where he/she/they do a* punch (into a)* punch (into a)* punch

# structure: move/string 

In [22]:
# python -m spacy train config.cfg --output ./output --paths.train ./train.spacy --paths.dev ./train.spacy 



In [23]:
from types import NoneType


entity_names = ['MOVE_TYPE', 'MOVE_HIT_PROPERTY', 'MOVE_BLOCK_PROPERTY', 'MOVE_PROPERTY', 'RANGE', 'LIMB', 'MOVE_STATE', 'SPECIFIC_INPUT', 'MOVE_NAME', 'STRING_DETAIL']
train_data = data['annotations']
#train_data = [tuple(i) for i in train_data]
new_list = []

# remove all NoneType data that may have been introduced
for x in train_data[:]:
    if isinstance(x, NoneType):
        train_data.remove(x)

# format the data into the required format
for i in range(0, len(train_data)):
    new_list = []
    tupled_data = tuple(train_data[i])
    for lists in train_data[i][1].get('entities'):
        new_list.append(tuple(lists))
    new_dict = {
        'entities': new_list
    }
    tupled_data = (train_data[i][0], new_dict)
    train_data[i] = tupled_data

print(train_data)
    

[('Low ', {'entities': [(0, 3, 'MOVE_HIT_PROPERTY')]}), ('Mid', {'entities': [(0, 3, 'MOVE_HIT_PROPERTY')]}), ('High', {'entities': [(0, 4, 'MOVE_HIT_PROPERTY')]}), ('Guard crush/unblockable', {'entities': [(0, 23, 'MOVE_HIT_PROPERTY')]}), ('Counterhit', {'entities': [(0, 10, 'MOVE_HIT_PROPERTY')]}), ('Tornado', {'entities': [(0, 7, 'MOVE_HIT_PROPERTY')]}), ('launch', {'entities': [(0, 6, 'MOVE_HIT_PROPERTY')]}), ('Mini combo ', {'entities': [(0, 10, 'MOVE_HIT_PROPERTY')]}), ('Flip over ', {'entities': [(0, 9, 'MOVE_HIT_PROPERTY')]}), ('Heat engager', {'entities': [(0, 12, 'MOVE_TYPE')]}), ('Parry', {'entities': [(0, 5, 'MOVE_TYPE')]}), ('String', {'entities': [(0, 6, 'MOVE_TYPE')]}), ('Power crush', {'entities': [(0, 11, 'MOVE_TYPE')]}), ('Stance ', {'entities': [(0, 6, 'MOVE_TYPE')]}), ('plus ', {'entities': [(0, 4, 'MOVE_BLOCK_PROPERTY')]}), ('minus ', {'entities': [(0, 5, 'MOVE_BLOCK_PROPERTY')]}), ('evasive', {'entities': [(0, 7, 'MOVE_PROPERTY')]}), ('transition', {'entities': [(

In [24]:
nlp = spacy.load("en_core_web_sm") # load other spacy model
db = DocBin() 

#format data into spacy format
for text, annot in train_data: 
    doc = nlp.make_doc(text) 
    ents = []
    for start, end, label in annot["entities"]: 
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents 
    db.add(doc) 

os.chdir(r'/home/dlin1765/repos/tekken-move-guesser')
db.to_disk("./train.spacy") 

In [25]:
# add entity ruler pipeline to the nlp
# ruler = EntityRuler(nlp, overwrite_ents=True)
# ruler = nlp.add_pipe("entity_ruler")
import re
# config = {
#    "phrase_matcher_attr": None,
#    "validate": True,
#    "overwrite_ents": True,
#    "ent_id_sep": "||",
# }
# nlpTest = spacy.load("en_core_web_sm")

#ruler = nlpTest.add_pipe("entity_ruler", config=config)

# ^(?:(?:[udbnf]{0,10}|([udbnf]*[\s,.+]*){0,10}|(sway)?|(dpd)?)?[\s,]*[1-4](?:\s*[,+-]\s*[1-4])*)+$|^((?:[1-4](?:\s*[,+-]\s*[1-4])*)+)$



# specificMoveNames = ['lethal sky rocket', 'jfsr', 'skyrocket', 'sky rocket', '10 hit', '10hit', 'ten hit']
# directionInputs = ['n', 'NOINPUT', 'neutral', 'f', 'forward', 'df', 'down forward', 'downforward', 'down-forward', 'd', 'down', 'db', 'downback', 'down back', 'down-back', 'b', 'back', 'ub', 'upback', 'up back', 'up-back', 'u', 'up', 'uf', 'up forward', 'upforward', 'up-forward', 'ss', 'sidestep', 'side step', 'qcf' 'quarter circle forward', 'quartercircleforward', 'quarter-circle-forward', 'qcb', 'quarter circle back', 'quarter-circle-back', 'quartercircleback', 'ff', 'forwardforward', 'f,f', 'f, f', 'fff', 'f,f,f', 'f, f, f', 'while running', 'wr','triple forward', 'forwardforwardforward', 'bf', 'back forward', 'backforward', 'bnf', 'b,n,f', 'b, n, f', 'back neutral forward', 'backneutralforward', 'back,neutral,forward', 'ws', 'while standing', 'while rising', 'w.s', 'w,s', 'w s', 'whilestanding', 'whilerising', 'fc', 'full crouch', 'fullcrouch', 'cd', 'crouch dash', 'crouchdash', 'bt', 'backturn', 'back turn', 'back-turn']
# delimiters = ['',' ', '+', ' + ', ' +', '+ ', '.', ' . ', ' .', '. ', ',', ' , ', ', ', ' ,', '-', ' - ', '- ', ' -',]
# allPossibleInputs = ['1', '2', '3', '4']

# patterns1 = [
#             {"TEXT": {"REGEX": "(?:(?:[udbnf]{0,10}|([udbnf]+[\s,.+]*){0,10}|(sway)?|(dpd)?)?[\s,]*[1-4](?:\s*[,+-]\s*[1-4])*)+|^((?:[1-4](?:\s*[,+-]\s*[1-4])*)+)"}}
#             ]

# patterns = [{"label": "SPECIFIC_MOVE", "pattern": [
#             {"TEXT": {"REGEX": "(?:(?:[udbnf]{0,10}|([udbnf]+[\s,.+]*){0,10}|(sway)?|(dpd)?)?[\s,]*[1-4](?:\s*[,+-]\s*[1-4])*)+|((?:[1-4](?:\s*[,+-]\s*[1-4])*)+)"}}
#             ]}]




#ruler.add_patterns([{"TEXT": {"REGEX": "(?:(?:[udbnf]{0,10}|([udbnf]+[\s,.+]*){0,10}|(sway)?|(dpd)?)?[\s,]*[1-4](?:\s*[,+-]\s*[1-4])*)+|^((?:[1-4](?:\s*[,+-]\s*[1-4])*)+)"}}])
#ruler.add_patterns(patterns)
#testDoc = nlpTest("this is a sentence about drag b 3,2, or written as b3,2")

@Language.component("specificMoves")
def specificMoves(doc):
    new_ents = []
    expression = "(?:(?:[udbnf]{0,10}|([udbnf]+[\s,.+]*){0,10}|(sway)?|(dpd)?|(ss)?)?[\s,]*[1-4](?:\s*[,+-]\s*[1-4])*)+|((?:[1-4](?:\s*[,+-]\s*[1-4])*)+)"
    for match in re.finditer(expression, doc.text):
        start, end = match.span()
        span = doc.char_span(start, end, label = "SPECIFIC_MOVE")
        if span is not None:
            new_ents.append(span)

    doc.ents = list(doc.ents) + new_ents
    return doc



In [26]:
import spacy.displacy
#adding a tagger and a parser to  

#nlp1 = spacy.load(r"/home/dlin1765/repos/tekken-move-guesser/output/model-best") 


nlp1 = spacy.load(r"/home/dlin1765/repos/tekken-move-guesser/output/model-last") 
# new model correctly finds all instances of specific moves
doc8 = nlp1("claudio b2,3,4")
doc9 = nlp1("claudio df2,3,4")
doc10 = nlp1("claudio df 2+3 with outer stuff too")

spacy.displacy.render(doc8, style="ent", jupyter=True)
spacy.displacy.render(doc9, style="ent", jupyter=True)
spacy.displacy.render(doc10, style="ent", jupyter=True)

# kind of messes up certain properties, like "three hit string" if it was 3 hit string it thinks its talking about the 3 MOVE
doc = nlp1("punch string that is safe on block")
doc1 = nlp1("high damage mid punch heat engager")
doc2 = nlp1("three hit string that starts with a low")
doc6 = nlp1("safe on block mid from stance")

spacy.displacy.render(doc, style="ent", jupyter=True) 
spacy.displacy.render(doc1, style="ent", jupyter=True) 
spacy.displacy.render(doc2, style="ent", jupyter=True) 
spacy.displacy.render(doc6, style="ent", jupyter=True)

In [27]:
# results 
# much better results than a few places (just the specific moves part)
# worse at general stuff



In [28]:
# does identify the stance 
doc3 = nlp1("mid poke into high that transition into a stance")
doc5 = nlp1("stance high homing move")
doc7 = nlp1("jumping ch launch low that goes into stance")
# doesnt identify the stance
doc4 = nlp1("high homing move from stance")

print("identifies the stance")
spacy.displacy.render(doc3, style="ent", jupyter=True)
spacy.displacy.render(doc7, style="ent", jupyter=True)
spacy.displacy.render(doc5, style="ent", jupyter=True)
print("does not identify the stance")
spacy.displacy.render(doc4, style="ent", jupyter=True)

identifies the stance


does not identify the stance
